In [117]:
# Теперь решаем задачу регрессии - предскажем цены на недвижимость. 
# Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)

import pandas as pd

In [118]:
data_or = pd.read_csv('train.csv')  
# len(data) = 1460

target = data_or['SalePrice']
data_or = data_or.drop('SalePrice', axis = 1)
data = data_or
data.shape

(1460, 80)

In [119]:
int64_feat = list(data_or.dtypes[data_or.dtypes=='int64'].index)
# None

cat_feat = list(data_or.dtypes[data_or.dtypes==object].index)
# Alley            91   non-null object      # MasVnrType       1452 non-null object
# BsmtQual         1423 non-null object      # BsmtCond         1423 non-null object
# BsmtExposure     1422 non-null object      # BsmtFinType1     1423 non-null object
# BsmtFinType2     1422 non-null object      # Electrical       1459 non-null object
# FireplaceQu      770  non-null object      # GarageType       1379 non-null object
# GarageFinish     1379 non-null object      # GarageQual       1379 non-null object
# GarageCond       1379 non-null object      # PoolQC           7    non-null object
# Fence            281  non-null object      # MiscFeature      54   non-null object

float64_feat = list(data_or.dtypes[data_or.dtypes=='float64'].index)
# LotFrontage    1201 non-null float64
# MasVnrArea     1452 non-null float64
# GarageYrBlt    1379 non-null float64

In [120]:
data[cat_feat] = data_or[cat_feat].fillna('nan')
# data[cat_feat].info()
data[float64_feat] = data_or[float64_feat].fillna(-999)
# data[float64_feat].info()

In [121]:
dummy_data_cat = pd.get_dummies(data[cat_feat], columns=cat_feat)
cat_nunique = data[cat_feat].nunique()
# len(cat_nunique) = 43
cat_feat_reduced = list(cat_nunique[cat_nunique<10].index)
# len(cat_feat_reduced) = 40

In [122]:
processed_data = pd.concat([data[int64_feat], data[float64_feat], dummy_data_cat],axis=1)

Linear Regression

In [123]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
train_X, test_X, train_y, test_y = train_test_split(processed_data, target, test_size = 0.3, random_state = 12)

In [124]:
print('train_X.shape = ', train_X.shape)
print('train_y.shape = ', train_y.shape)
print('test_X.shape = ', test_X.shape)
print('test_y.shape = ', test_y.shape)

train_X.shape =  (1022, 305)
train_y.shape =  (1022,)
test_X.shape =  (438, 305)
test_y.shape =  (438,)


In [125]:
from sklearn.decomposition import PCA

In [126]:
pca = PCA(n_components=2)
ppl = pca.fit_transform(train_X)
pca.explained_variance_ratio_.sum()

0.982990449083399

In [127]:
model = LinearRegression()
model.fit(train_X,train_y)
pred_y = model.predict(test_X)
print('     AVG_cost = ',test_y.mean())
print('pred_AVG_cost = ',pred_y.mean())

     AVG_cost =  182380.0
pred_AVG_cost =  179754.5249522874


Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей

In [128]:
from sklearn.model_selection import cross_val_score  
from sklearn.metrics import r2_score

In [129]:
regressor = LinearRegression()

In [130]:
all_accuracies = cross_val_score(estimator=regressor, X=processed_data, y=target, cv=10)  

In [131]:
all_accuracies.mean()

0.5985854835139343

In [132]:
from sklearn.metrics import r2_score
regressor1 = LinearRegression()

In [133]:
regressor1.fit(train_X,train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [134]:
rss = np.sum((regressor1.predict(processed_data) - target) ** 2)
print("RSS:", rss)
print("R^2:", r2_score(regressor1.predict(test_X), test_y))

RSS: 2537739790093.2725
R^2: 0.5264346515941885


In [135]:
# Построить случайный лес, вывести важность признаков
    
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=10, max_depth=5, min_samples_leaf=20, max_features=0.5, n_jobs=-1)
clf_rf.fit(train_X, train_y)   

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [136]:
imp = pd.Series(clf_rf.feature_importances_)
imp = imp.sort_values(ascending=False)
imp = imp[imp.values>0.01]
print(train_X.columns[imp.index])

Index(['2ndFlrSF', '1stFlrSF', 'GrLivArea', 'LotArea', 'TotalBsmtSF',
       'YearBuilt', 'YearRemodAdd', 'WoodDeckSF', 'GarageArea', 'OpenPorchSF',
       'FullBath', 'GarageType_Attchd', 'OverallQual', 'BsmtUnfSF',
       'Neighborhood_NWAmes', 'BsmtFinSF1', 'LandContour_Bnk',
       'GarageFinish_Fin', 'Id', 'BsmtFinSF2', 'GarageCars', 'LotFrontage',
       'TotRmsAbvGrd', 'HeatingQC_TA', 'Foundation_PConc', 'GarageFinish_RFn',
       'KitchenQual_Gd', 'LandContour_Lvl'],
      dtype='object')


In [137]:
# Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
import numpy as np

In [138]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, shuffle=True, noise=0.4, random_state=123)

In [139]:
clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)

In [140]:
for clf, label in zip([clf1, clf2, clf3, sclf], 
                      ['KNN', 
                       'Random Forest', 
                       'Naive Bayes',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, X, y, 
                                              cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

Accuracy: 0.81 (+/- 0.01) [KNN]
Accuracy: 0.86 (+/- 0.01) [Random Forest]
Accuracy: 0.83 (+/- 0.02) [Naive Bayes]
Accuracy: 0.81 (+/- 0.02) [StackingClassifier]


In [141]:
# Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии

params = {'kneighborsclassifier__n_neighbors': [1, 5],
          'randomforestclassifier__n_estimators': [10, 50],
          'meta-logisticregression__C': [0.1, 10.0]}

grid = GridSearchCV(estimator=sclf, param_grid=params, cv=10,refit=True)
grid.fit(X, y)

cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r"
          % (grid.cv_results_[cv_keys[0]][r],
             grid.cv_results_[cv_keys[1]][r] / 2.0,
             grid.cv_results_[cv_keys[2]][r]))

print('Best parameters: %s' % grid.best_params_)
print('Accuracy: %.2f' % grid.best_score_)

0.828 +/- 0.02 {'kneighborsclassifier__n_neighbors': 1, 'meta-logisticregression__C': 0.1, 'randomforestclassifier__n_estimators': 10}
0.824 +/- 0.02 {'kneighborsclassifier__n_neighbors': 1, 'meta-logisticregression__C': 0.1, 'randomforestclassifier__n_estimators': 50}
0.808 +/- 0.02 {'kneighborsclassifier__n_neighbors': 1, 'meta-logisticregression__C': 10.0, 'randomforestclassifier__n_estimators': 10}
0.826 +/- 0.02 {'kneighborsclassifier__n_neighbors': 1, 'meta-logisticregression__C': 10.0, 'randomforestclassifier__n_estimators': 50}
0.852 +/- 0.01 {'kneighborsclassifier__n_neighbors': 5, 'meta-logisticregression__C': 0.1, 'randomforestclassifier__n_estimators': 10}
0.854 +/- 0.01 {'kneighborsclassifier__n_neighbors': 5, 'meta-logisticregression__C': 0.1, 'randomforestclassifier__n_estimators': 50}
0.852 +/- 0.01 {'kneighborsclassifier__n_neighbors': 5, 'meta-logisticregression__C': 10.0, 'randomforestclassifier__n_estimators': 10}
0.854 +/- 0.01 {'kneighborsclassifier__n_neighbors':